In [1]:
import tensorflow as tf
import pyttsx3
import speech_recognition as sr
from importnb import Notebook
import pickle
with Notebook():
    from English_Spanish import NeuralMachineTranslation
    from Spanish_English import NeuralMachineTranslation

In [ ]:
en_sp_model = tf.keras.models.load_model('./en_sp_model.h5')
sp_en_model = tf.keras.models.load_model('./sp_en_model.h5')

# Load tokenizers
with open("en_sp_input_tokenizer.pkl", "rb") as f:
    en_sp_input_tokenizer = pickle.load(f)

with open("en_sp_output_tokenizer.pkl", "rb") as f:
    en_sp_output_tokenizer = pickle.load(f)

with open("sp_en_input_tokenizer.pkl", "rb") as f:
    sp_en_input_tokenizer = pickle.load(f)

with open("sp_en_output_tokenizer.pkl", "rb") as f:
    sp_en_output_tokenizer = pickle.load(f)

In [5]:
def translate(model, input_sentence, input_tokenizer, output_tokenizer, wrapper_cls):
    # Wrap the model with tokenizers
    nmt_model = wrapper_cls(max_sequence_length=5)
    nmt_model.model = model
    nmt_model.input_tokenizer = input_tokenizer
    nmt_model.output_tokenizer = output_tokenizer

    # Translate the sentence
    return nmt_model.translate(input_sentence)


def translate_en_sp(input_sentence):
    return translate(
        en_sp_model, input_sentence, en_sp_input_tokenizer, en_sp_output_tokenizer, NeuralMachineTranslation
    )


def translate_sp_en(input_sentence):
    return translate(
        sp_en_model, input_sentence, sp_en_input_tokenizer, sp_en_output_tokenizer, NeuralMachineTranslation
    )

In [6]:
def recognize_speech(language="en-US"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print(f"Listening for speech in {language}...")
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)
            text = recognizer.recognize_google(audio, language=language)
            print(f"Recognized Text ({language}): {text}")
            return text
        except sr.UnknownValueError:
            print("Speech not understood.")
        except sr.RequestError:
            print("Error with the speech recognition service.")
        except Exception as e:
            print(f"Error: {e}")
        return ""

In [7]:
import pyttsx3

def text_to_speech(text, language="en"):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    voices = engine.getProperty('voices')
    if language == "es":
        engine.setProperty('voice', voices[1].id)  # Adjust for Spanish
    else:
        engine.setProperty('voice', voices[0].id)  # Adjust for English
    engine.say(text)
    engine.runAndWait()


In [8]:
def real_time_translation():
    try:
        while True:
            print("\n--- English to Spanish ---")
            # Modify function calls
            english_text = recognize_speech("en-US")
            if english_text:
                spanish_translation = translate_en_sp(english_text)
                text_to_speech(spanish_translation, "es")

            
            print("\n--- Spanish to English ---")
            spanish_text = recognize_speech("es-ES")
            if spanish_text:
                english_translation = translate_sp_en(spanish_text)
                text_to_speech(english_translation, "en")
    except KeyboardInterrupt:
        print("\nExiting the program...")


In [9]:
if __name__ == "__main__":
    real_time_translation()


--- English to Spanish ---
Listening for speech in en-US...
Recognized Text (en-US): how r u
1/1 [==============================] - 0s 22ms/step

--- Spanish to English ---
Listening for speech in es-ES...
Speech not understood.

--- English to Spanish ---
Listening for speech in en-US...
Recognized Text (en-US): hu r u
1/1 [==============================] - 0s 19ms/step

--- Spanish to English ---
Listening for speech in es-ES...

Exiting the program...
